In [433]:
import pandas as pd
import numpy as np

# Lectura de datos

In [476]:
data = pd.read_csv("data/AmesHousing.csv")
ATTR = ["1st Flr SF", "2nd Flr SF", "Bedroom AbvGr", "Bsmt Full Bath", "Bsmt Unf SF", "BsmtFin SF 1", "Fireplaces", 
        "Full Bath", "Garage Area", "Garage Cars", "Garage Yr Blt", "Gr Liv Area", "Half Bath", "Open Porch SF", 
        "Overall Cond", "Overall Qual", "Total Bsmt SF", "TotRms AbvGrd", "Wood Deck SF", "Year Built", 
        "Year Remod/Add","Bsmt Cond", "Bsmt Exposure", "Bsmt Qual", "BsmtFin Type 1", "Central Air", "Electrical", 
        "Exter Qual", "Foundation", "Garage Finish", "Garage Type", "Heating QC", "Kitchen Qual", "Lot Shape", 
        "MS Zoning", "Paved Drive", "Roof Style", "Sale Condition", "Sale Type", "SalePrice"]

In [477]:
data = data[data['Garage Yr Blt']!=2207] #eliminamos outlier año=2207

Hayamos el número de valores ausentes en los datos, considerando solo los atributos seleccionados (ATTR)

In [478]:
sumas = data.isna().sum()
for c in ATTR:
    if(sumas[c] > 0):
        print("%s contiene %d valores NaN" % (c, sumas[c]))

Bsmt Full Bath contiene 2 valores NaN
Bsmt Unf SF contiene 1 valores NaN
BsmtFin SF 1 contiene 1 valores NaN
Garage Area contiene 1 valores NaN
Garage Cars contiene 1 valores NaN
Garage Yr Blt contiene 159 valores NaN
Total Bsmt SF contiene 1 valores NaN
Bsmt Cond contiene 80 valores NaN
Bsmt Exposure contiene 83 valores NaN
Bsmt Qual contiene 80 valores NaN
BsmtFin Type 1 contiene 80 valores NaN
Electrical contiene 1 valores NaN
Garage Finish contiene 159 valores NaN
Garage Type contiene 157 valores NaN


# Valores ausentes de Garaje

Hemos observado que muchos de los atributos que tienen que ver con el garaje contienen un número de **NaN** similar entre ellos. La hipótesis es que se trata de aquellas casas sin garaje (Garage Area == 0). Comprobamos dicha hipótesis para los atributos seleccionados para la regresión

`no_garages` es un DataFrame que contiene aquellas casas sin garaje. Queremos saber si los atributos *Garage Yr Blt*, *Garage Finish* y *Garage Type* tienen siempre valor **NaN** cuando la casa no tiene garaje.

In [479]:
no_garages = data[data["Garage Area"] == 0]

In [480]:
no_garages["Garage Yr Blt"].count()

0

In [481]:
no_garages["Garage Finish"].count()

0

In [482]:
no_garages["Garage Type"].count()

0

Todos los `count()` son 0, por lo que estos atributos toman el valor **NaN** siempre que el área de garaje sea 0

In [483]:
null_yr_blt = data[data["Garage Yr Blt"].isnull()]
null_gr_finish = data[data["Garage Finish"].isnull()]
null_gr_type = data[data["Garage Type"].isnull()]

También queremos conocer si cualquiera de estos atributos toma valor **NaN** cuando la casa tiene garaje (Garage Area > 0)

In [484]:
null_yr_blt["Garage Area"].max()

360.0

In [485]:
null_yr_blt[null_yr_blt["Garage Area"]>0]

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
1356,1357,903426160,60,RM,57.0,8094,Pave,Grvl,Reg,Lvl,...,0,NaN,MnPrv,Shed,1000,9,2008,WD,Normal,160000


In [486]:
null_gr_finish["Garage Area"].max()

360.0

In [487]:
null_gr_finish[null_gr_finish["Garage Area"]>0]

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
1356,1357,903426160,60,RM,57.0,8094,Pave,Grvl,Reg,Lvl,...,0,NaN,MnPrv,Shed,1000,9,2008,WD,Normal,160000


Existe una instancia con área de garaje igual a 360 sq.feet cuyo Garage Finish y Garage Yr Blt es **NaN**. La eliminamos utilizando su PID.

In [488]:
data = data[data.PID != 903426160]

In [489]:
null_gr_type["Garage Area"].max()

0.0

Tras esto, ya podemos rellenar los valores **NaN** en los atributos categóricos con un valor "No Garage"

In [490]:
data.loc[data["Garage Finish"].isnull(),"Garage Finish"] = "No Garage"
data.loc[data["Garage Type"].isnull(),"Garage Type"] = "No Garage"

Respecto al atributo numérico "Garage Yr Blt", por el momento se decide no utilizar dicho atributo, aunque se considerará para posibles interacciones con el atributo Garage Area.

# Valores ausentes de Sótano

Además de un fenómeno similar al anterior, queremos comprobar que BsmtFin SF 1 + BsmtFin SF 2 + Bsmt Unf SF sea igual a Total Bsmt SF en todas las instancias

In [491]:
(data["BsmtFin SF 1"] + data["BsmtFin SF 2"] + data["Bsmt Unf SF"] == data["Total Bsmt SF"]).all()

False

Como se puede observar, esto no es así para todas las instancias, hay una potencialmente incorrecta. La eliminamos.

In [492]:
data[~(data["BsmtFin SF 1"] + data["BsmtFin SF 2"] + data["Bsmt Unf SF"] == data["Total Bsmt SF"])]

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
1341,1342,903230120,20,RM,99.0,5940,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,4,2008,ConLD,Abnorml,79000


In [493]:
data = data[data.PID != 903230120]

Se ha observado un fenómeno similar con los atributos relacionados con el sótano. La hipótesis es que se trata de aquellas casas sin sótano. En este caso hay sótano terminado y no terminado, por lo que son casas con Total Bsmt SF == 0. Comprobamos dicha hipótesis para los atributos seleccionados para la regresión

`no_sotano` es un DataFrame que contiene aquellas casas sin sótano. Queremos saber si los atributos *Bsmt Full Bath*, *Bsmt Cond*, *Bsmt Exposure*, *BsmtFin Type 1* y *Bsmt Qual* tienen siempre valor **NaN** cuando la casa no tiene sótano.

In [494]:
no_sotano = data[data["Total Bsmt SF"] == 0]

In [495]:
no_sotano["Bsmt Full Bath"].count()

78

In [496]:
no_sotano["Bsmt Full Bath"].sum()

0.0

In [497]:
no_sotano[no_sotano["Bsmt Full Bath"].isna()]

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
1497,1498,908154080,20,RL,123.0,47007,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,7,2008,WD,Normal,284700


En el caso de `Bsmt Full Bath`, el valor de área 0 siempre resulta en un valor de 0 baños en el sótano excepto por una instancia con valor **NaN** en *Bsmt Full Bath*.

La rellenamos con un 0.

In [498]:
data.loc[data.PID==908154080,"Bsmt Full Bath"] = 0

In [499]:
no_sotano["Bsmt Cond"].count()

0

In [500]:
no_sotano["Bsmt Exposure"].count()

0

In [501]:
no_sotano["BsmtFin Type 1"].count()

0

Todos los `count()` son 0, por lo que estos atributos toman el valor **NaN** siempre que el área de sótano sea 0

In [502]:
null_bsmt_cond = data[data["Bsmt Cond"].isnull()]
null_bsmt_exposure = data[data["Bsmt Exposure"].isnull()]
null_bsmt_type = data[data["BsmtFin Type 1"].isnull()]

También queremos conocer si cualquiera de estos atributos toma valor **NaN** cuando la casa tiene sótano (Total Bsmt SF > 0)

In [503]:
null_baths["Total Bsmt SF"].max()

0.0

In [504]:
null_bsmt_cond["Total Bsmt SF"].max()

0.0

In [505]:
null_bsmt_exposure["Total Bsmt SF"].max()

1595.0

In [506]:
null_bsmt_exposure[null_bsmt_exposure["Total Bsmt SF"]>0]

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
66,67,528445060,20,RL,73.0,8987,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,221500
1796,1797,528458090,60,FV,81.0,10411,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,7,2007,New,Partial,212109
2779,2780,907194130,60,RL,65.0,14006,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,2,2006,WD,Normal,192500


Existen tres instancias con área de sótano mayor que 0 cuyo Bsmt Exposure es **NaN**. Las eliminamos utilizando sus PIDs.

In [507]:
data = data[data.PID != 528445060]
data = data[data.PID != 528458090]
data = data[data.PID != 907194130]

In [508]:
null_bsmt_type["Total Bsmt SF"].max()

0.0

Tras esto, ya podemos rellenar los valores **NaN** en los atributos categóricos con un valor "No Basement"

In [509]:
data.loc[data["Bsmt Cond"].isnull(),"Bsmt Cond"] = "No Basement"
data.loc[data["Bsmt Exposure"].isnull(),"Bsmt Exposure"] = "No Basement"
data.loc[data["Bsmt Qual"].isnull(),"Bsmt Qual"] = "No Basement"
data.loc[data["BsmtFin Type 1"].isnull(),"BsmtFin Type 1"] = "No Basement"

# Final

In [510]:
sumas = data.isna().sum()
ATTR.remove("Garage Yr Blt")
for c in ATTR:
    if(sumas[c] > 0):
        print("%s contiene %d valores NaN" % (c, sumas[c]))

Garage Area contiene 1 valores NaN
Garage Cars contiene 1 valores NaN
Electrical contiene 1 valores NaN


Podemos eliminar las instancias con `Garage Area`, `Garage Cars` y `Electrical` ausentes, ya que son muy pocas

In [511]:
data[data["Garage Area"].isna()]

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
2236,2237,910201180,70,RM,50.0,9060,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,3,2007,WD,Alloca,150909


In [512]:
data[data["Garage Cars"].isna()]

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
2236,2237,910201180,70,RM,50.0,9060,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,3,2007,WD,Alloca,150909


In [513]:
data[data["Electrical"].isna()]

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
1577,1578,916386080,80,RL,73.0,9735,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,5,2008,WD,Normal,167500


In [514]:
data = data[data.PID != 910201180]
data = data[data.PID != 916386080]

# Guardado de fichero

Tras la modificación, guardamos el fichero `data.csv` modificado

In [515]:
data.to_csv("data/AmesHousing_modified.csv",index=False)